In [2]:
from pdb import set_trace
import os
import spacy
import numpy as np
import pandas as pd
from tqdm import tqdm
import multiprocessing as mp
import warnings
import inspect
import types
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM


/home/y-guo/miniconda3/envs/flexattention/lib/python3.10/site-packages/transformers/utils/hub.py:111: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(


In [3]:
from constants import MODEL_PATHs

In [4]:
nlp = None
num_parts = 8


In [5]:
    from dataset import MyriadLamaDataset
    dataset = MyriadLamaDataset(model_name="llama3.2_3b_it")
    

Dataset already exists at /net/tokyo100-10g/data/str01_01/y-guo/datasets/myriadlama/paraphrases_dataset. Loading from disk.


In [6]:
   dataloader = dataset.get_dataloader(batch_size=2, shuffle=False)

In [7]:
args = types.SimpleNamespace()
args.model = "llama3.2_3b_it"  # IGNORE
args.num_paraphrases = 3  # IGNORE

In [8]:
    model_path = MODEL_PATHs.get(args.model, args.model)

In [9]:
import myriadlama_flex_attention_generate

✅ FlexAttention is available


In [10]:
    few_shot_examples = myriadlama_flex_attention_generate.get_few_shot_examples_with_paraphrases(
        dataset, 
        k=5, 
        num_fs_paraphrases=args.num_paraphrases,
        seed=42
    )

In [11]:
    instruction = dataset.instruction

In [12]:
few_shot_examples

[{'paraphrases': ['[MASK] is the higher-level concept of adit.',
   'adit is also necessarily a [MASK].',
   '[MASK] is the superclass of adit.'],
  'answer': 'tunnel'},
 {'paraphrases': ['Marty Ehrlich became known as a [MASK] player.',
   'Marty Ehrlich is a [MASK] player.',
   'Marty Ehrlich plays [MASK].'],
  'answer': 'saxophone'},
 {'paraphrases': ['Officially, the people living in Basel use the language [MASK] for communication.',
   '[MASK] recognizes Basel as its official language.',
   'What language is officially spoken in Basel? [MASK].'],
  'answer': 'German'},
 {'paraphrases': ['[MASK] consists of Mount Makiling.',
   '[MASK] encompasses Mount Makiling as its element.',
   'Mount Makiling represents a segment of [MASK].'],
  'answer': 'Luzon'},
 {'paraphrases': ['Iowa and [MASK] are neighboring countries.',
   'You can go through Iowa to reach [MASK].',
   'Which country is adjacent to Iowa? [MASK].'],
  'answer': 'Wisconsin'}]

In [13]:
import pdb

In [14]:
dataloader

In [15]:
            batch_predictions = []
            batch_generations = []
            batch_templates = []
            batch_debug_database = []
            batch_debug_prompt = []
            batch_debug_mask = []
            batch_debug_output = []

In [37]:
for a, b, c in dataloader:
    print(a)
    print(b)
    break

['5d09424876ba7f4-15e8746c51e1d4c', 'fd915def246a0be-1df91599c979dd7']
[['NBC', 'National Broadcasting Company'], ['physics', 'physical', 'physic', 'general physics', 'physical science']]


In [16]:
from transformers.models.llama.modeling_llama import LlamaModel


In [17]:
instruction

'Based on the context, predict the [MASK] in the sentence in one word. Do NOT use [MASK] in your answer.'

In [18]:
few_shot_examples = myriadlama_flex_attention_generate.get_few_shot_examples_with_paraphrases(
        dataset, 
        k=5, 
        num_fs_paraphrases=5,
        seed=42
    )

In [19]:
few_shot_examples

[{'paraphrases': ['[MASK] is the higher-level concept of adit.',
   'adit is also necessarily a [MASK].',
   '[MASK] is the superclass of adit.',
   'adit is a subclass of [MASK].',
   'Of which class is adit a subclass? [MASK].'],
  'answer': 'tunnel'},
 {'paraphrases': ['Marty Ehrlich became known as a [MASK] player.',
   'Marty Ehrlich is a [MASK] player.',
   'Marty Ehrlich plays [MASK].',
   'What musical instrument does Marty Ehrlich play? [MASK].',
   'Marty Ehrlich can perform on the [MASK].'],
  'answer': 'saxophone'},
 {'paraphrases': ['Officially, the people living in Basel use the language [MASK] for communication.',
   '[MASK] recognizes Basel as its official language.',
   'What language is officially spoken in Basel? [MASK].',
   'The official language of Basel is [MASK].',
   '[MASK] is spoken as an official language by people in Basel.'],
  'answer': 'German'},
 {'paraphrases': ['[MASK] consists of Mount Makiling.',
   '[MASK] encompasses Mount Makiling as its element.

In [20]:
uids, answers, all_paraphrases = next(iter(dataloader))

In [21]:
uids

['5d09424876ba7f4-15e8746c51e1d4c', 'fd915def246a0be-1df91599c979dd7']

In [22]:
i, paraphrases = next(enumerate((zip(*all_paraphrases))))

In [23]:
paraphrases

('This Is Your Life premiered on the network [MASK].',
 '[MASK] is the first air channel of This Is Your Life.',
 'Which service originally aired This Is Your Life? [MASK].',
 'This Is Your Life was originally aired on [MASK].',
 'This Is Your Life was initially launched on [MASK].',
 'The first episode of This Is Your Life aired on [MASK].',
 '[MASK] had the distinction of being the first to air This Is Your Life.',
 'This Is Your Life was first unveiled on [MASK].',
 'This Is Your Life was initially shown on [MASK].',
 'The television show This Is Your Life had its first airing on [MASK].')

In [24]:
all_templates = list(paraphrases)
selected_templates = all_templates[:5]

In [25]:
len(instruction)

103

In [38]:
prompt_parts = myriadlama_flex_attention_generate.construct_prompt_new_format(
    instruction,
    few_shot_examples,
    selected_templates  # Pass ALL paraphrases
    )

In [31]:
prompt_parts

'Based on the context, predict the [MASK] in the sentence in one word. Do NOT use [MASK] in your answer.\n\nQ: [MASK] is the higher-level concept of adit.\nQ: adit is also necessarily a [MASK].\nQ: [MASK] is the superclass of adit.\nQ: adit is a subclass of [MASK].\nQ: Of which class is adit a subclass? [MASK].\nA: tunnel\n\nQ: Marty Ehrlich became known as a [MASK] player.\nQ: Marty Ehrlich is a [MASK] player.\nQ: Marty Ehrlich plays [MASK].\nQ: What musical instrument does Marty Ehrlich play? [MASK].\nQ: Marty Ehrlich can perform on the [MASK].\nA: saxophone\n\nQ: Officially, the people living in Basel use the language [MASK] for communication.\nQ: [MASK] recognizes Basel as its official language.\nQ: What language is officially spoken in Basel? [MASK].\nQ: The official language of Basel is [MASK].\nQ: [MASK] is spoken as an official language by people in Basel.\nA: German\n\nQ: [MASK] consists of Mount Makiling.\nQ: [MASK] encompasses Mount Makiling as its element.\nQ: Mount Makilin

In [39]:
prompt_parts[0]

'Based on the context, predict the [MASK] in the sentence in one word. Do NOT use [MASK] in your answer.\n\nQ: [MASK] is the higher-level concept of adit. A: tunnel\nQ: Marty Ehrlich became known as a [MASK] player. A: saxophone\nQ: Officially, the people living in Basel use the language [MASK] for communication. A: German\nQ: [MASK] consists of Mount Makiling. A: Luzon\nQ: Iowa and [MASK] are neighboring countries. A: Wisconsin\n'

In [36]:
import importlib
importlib.reload(myriadlama_flex_attention_generate)

✅ FlexAttention is available


<module 'myriadlama_flex_attention_generate' from '/home/y-guo/self-ensemble/self-ensemble/myriadlama_flex_attention_generate.py'>